Hücre 1: Kütüphaneleri Kuralım

In [9]:
# Gerekli kütüphaneleri import edelim
from youtube_transcript_api import YouTubeTranscriptApi
import chromadb
from chromadb.utils import embedding_functions
from langchain_text_splitters import RecursiveCharacterTextSplitter
import ollama
import re
from datetime import timedelta

print("✅ Tüm kütüphaneler yüklendi!")
print("🎥 YouTube Video Asistanı hazırlanıyor...")

✅ Tüm kütüphaneler yüklendi!
🎥 YouTube Video Asistanı hazırlanıyor...


Hücre 2: YouTube Video ID'sini Çekme Fonksiyonu

In [4]:
def get_video_id(url):
    """
    YouTube URL'sinden video ID'sini çıkar
    """
    # Farklı YouTube URL formatları
    patterns = [
        r'(?:https?:\/\/)?(?:www\.)?youtube\.com\/watch\?v=([a-zA-Z0-9_-]+)',
        r'(?:https?:\/\/)?(?:www\.)?youtu\.be\/([a-zA-Z0-9_-]+)',
        r'(?:https?:\/\/)?(?:www\.)?youtube\.com\/embed\/([a-zA-Z0-9_-]+)'
    ]
    
    for pattern in patterns:
        match = re.search(pattern, url)
        if match:
            return match.group(1)
    
    # Eğer sadece ID verilmişse direkt döndür
    if len(url) == 11:
        return url
    
    return None

# Test edelim
test_urls = [
    "https://www.youtube.com/watch?v=kqtD5dpn9C8",
    "https://youtu.be/kqtD5dpn9C8",
    "kqtD5dpn9C8"
]

print("🧪 Test:")
for url in test_urls:
    video_id = get_video_id(url)
    print(f"   URL: {url[:40]}... → ID: {video_id}")

🧪 Test:
   URL: https://www.youtube.com/watch?v=kqtD5dpn... → ID: kqtD5dpn9C8
   URL: https://youtu.be/kqtD5dpn9C8... → ID: kqtD5dpn9C8
   URL: kqtD5dpn9C8... → ID: kqtD5dpn9C8


Hücre 3: Transcript Çekme Fonksiyonu

In [22]:
def get_transcript(video_id):
    """
    YouTube video transcript'ini çek
    """
    try:
        print(f"📥 Video ID: {video_id}")
        print("⏳ Transcript çekiliyor...")
        
        # API instance oluştur
        api = YouTubeTranscriptApi()
        
        # Transcript'i çek (önce Türkçe dene, yoksa İngilizce)
        try:
            transcript = api.fetch(video_id, languages=['tr'])
            dil = "Türkçe"
        except:
            transcript = api.fetch(video_id, languages=['en'])
            dil = "İngilizce"
        
        # Transcript'i birleştir
        full_text = ""
        timestamps = []
        
        for entry in transcript:
            full_text += entry.text + " "
            timestamps.append({
                'text': entry.text,
                'start': entry.start,
                'duration': entry.duration
            })
        
        # Video süresini hesapla (son entry'den)
        last_entry = timestamps[-1]
        toplam_sure = last_entry['start'] + last_entry['duration']
        sure_dk = int(toplam_sure // 60)
        sure_sn = int(toplam_sure % 60)
        
        print(f"✅ Transcript çekildi!")
        print(f"   📝 Dil: {dil}")
        print(f"   ⏱️  Video süresi: {sure_dk} dakika {sure_sn} saniye")
        print(f"   📊 Toplam kelime: {len(full_text.split())}")
        print(f"   📄 Karakter sayısı: {len(full_text)}")
        
        return full_text, timestamps
        
    except Exception as e:
        import traceback
        print(f"❌ Hata: {str(e)}")
        print(f"\n🔍 Detay:\n{traceback.format_exc()}")
        print("\n💡 Muhtemel nedenler:")
        print("   • Video altyazısı yok")
        print("   • Video ID yanlış")
        print("   • Video gizli/silinmiş")
        return None, None

print("✅ Fonksiyon hazır!")

✅ Fonksiyon hazır!


Hücre 4: Test Edelim!

In [23]:
# TEST VİDEO (Python tutorial)
# Bu video altyazılı olduğu için çalışacak
test_video = "https://www.youtube.com/watch?v=gbQnd-J-DMw"

video_id = get_video_id(test_video)
transcript_text, timestamps = get_transcript(video_id)

# İlk 500 karakteri görelim
if transcript_text:
    print("\n" + "=" * 70)
    print("📄 TRANSCRIPT ÖRNEĞİ (İlk 500 karakter):")
    print("=" * 70)
    print(transcript_text[:500])
    print("...")


📥 Video ID: gbQnd-J-DMw
⏳ Transcript çekiliyor...
✅ Transcript çekildi!
   📝 Dil: Türkçe
   ⏱️  Video süresi: 14 dakika 46 saniye
   📊 Toplam kelime: 464
   📄 Karakter sayısı: 3236

📄 TRANSCRIPT ÖRNEĞİ (İlk 500 karakter):
[Müzik] Evlat ile devlet [Müzik] terazinin iki kefesinde durur ise hangi yürek sarsılmaz? [Müzik] [Müzik] Bir yandan baba ocağının ateşı, öte yandan cihanın bekası. Hangisi yakar, hangisi yaşatır? Mevi gördüm ya Resulallah. [Müzik] Ölüm kimin alın yazısıdır? Öldürmek kimin kaderidir? Genç aslan her zaman mı yaşlı aslanın ensesinde olur? [Müzik] Kan ya evlat üçündür ya da devlet üçündür. Lakin her damlası, her damlası yüreğe ağır kalır. [Müzik] [Alkış] [Müzik] [Müzik] Kapı bend olur, yiğitler taş
...


Hücre 5: Transcript'i Chunking Yapalım

In [24]:
# Önce transcript'in başarıyla çekildiğinden emin olalım
if transcript_text is None:
    print("❌ Transcript çekilemedi! Önce yukarıdaki hücreyi çalıştırıp transcript'i başarıyla çekin.")
else:
    # Text splitter
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,        # Her chunk 500 karakter
        chunk_overlap=100,     # 100 karakter overlap
        separators=["\n\n", "\n", ". ", " ", ""]
    )

    # Chunk'lara böl
    chunks = text_splitter.split_text(transcript_text)

    print(f"✂️  Transcript {len(chunks)} chunk'a bölündü")
    print(f"📏 Ortalama chunk boyutu: {sum(len(c) for c in chunks) / len(chunks):.0f} karakter")

    # İlk 3 chunk'ı görelim
    print("\n" + "=" * 70)
    print("📚 İLK 3 CHUNK:")
    print("=" * 70)

    for i in range(min(3, len(chunks))):
        print(f"\n--- CHUNK {i+1} ({len(chunks[i])} karakter) ---")
        print(chunks[i])

✂️  Transcript 9 chunk'a bölündü
📏 Ortalama chunk boyutu: 429 karakter

📚 İLK 3 CHUNK:

--- CHUNK 1 (438 karakter) ---
[Müzik] Evlat ile devlet [Müzik] terazinin iki kefesinde durur ise hangi yürek sarsılmaz? [Müzik] [Müzik] Bir yandan baba ocağının ateşı, öte yandan cihanın bekası. Hangisi yakar, hangisi yaşatır? Mevi gördüm ya Resulallah. [Müzik] Ölüm kimin alın yazısıdır? Öldürmek kimin kaderidir? Genç aslan her zaman mı yaşlı aslanın ensesinde olur? [Müzik] Kan ya evlat üçündür ya da devlet üçündür. Lakin her damlası, her damlası yüreğe ağır kalır

--- CHUNK 2 (498 karakter) ---
. Lakin her damlası, her damlası yüreğe ağır kalır. [Müzik] [Alkış] [Müzik] [Müzik] Kapı bend olur, yiğitler taş kesilir. Lakin geçecek olan yine devlet hükmüdür. [Müzik] Devlet iradesi kapıda duran yiğitler gibi. Dik durmalı, eğilmemeli, bükülmemeli. He. Sultanım, ferman ile huzuruma çağırmış iken karşıma bir asi olarak mı çıkmalıydın? >> Sultanım >> oğullar büyüttük. Hükmümüzü cihana salsın Dehu. Ama oğlu

Hücre 6: ChromaDB'ye Kaydetme

In [25]:
print("🧠 Embedding modeli yükleniyor...")

# Embedding fonksiyonu
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="all-MiniLM-L6-v2"
)

# ChromaDB client
client = chromadb.Client()

# Eski collection'ı sil
try:
    client.delete_collection("youtube_db")
except:
    pass

# Yeni collection
collection = client.create_collection(
    name="youtube_db",
    embedding_function=sentence_transformer_ef
)

print("✅ Vector Database hazır!")

# Chunk'ları ekle
print(f"💾 {len(chunks)} chunk ekleniyor...")

collection.add(
    documents=chunks,
    ids=[f"chunk_{i}" for i in range(len(chunks))],
    metadatas=[{"chunk_id": i, "video_id": video_id} for i in range(len(chunks))]
)

print(f"✅ {len(chunks)} chunk başarıyla eklendi!")
print("🎉 Sistem hazır!")

🧠 Embedding modeli yükleniyor...
✅ Vector Database hazır!
💾 9 chunk ekleniyor...
✅ 9 chunk başarıyla eklendi!
🎉 Sistem hazır!


Hücre 7: Test Araması

In [26]:
# Test sorusu
test_soru = "What is this video about?"

print(f"🔍 Soru: {test_soru}\n")
print("📚 İlgili chunk'lar aranıyor...\n")

results = collection.query(
    query_texts=[test_soru],
    n_results=3
)

print("=" * 70)
print("BULUNAN CHUNK'LAR:")
print("=" * 70)

for i, (doc, dist) in enumerate(zip(results['documents'][0], results['distances'][0])):
    print(f"\n🏆 Chunk {i+1} - Benzerlik: {1-dist:.3f}")
    print("-" * 70)
    print(doc)

🔍 Soru: What is this video about?

📚 İlgili chunk'lar aranıyor...

BULUNAN CHUNK'LAR:

🏆 Chunk 1 - Benzerlik: 0.153
----------------------------------------------------------------------
. [Müzik] Sen şu kapıdan sağ salim girdin ya. Başkaca nimet istemem. Sultan baban seni bağışladı ya. Allah iki cihanda razı olsun ondan. >> Yalnızca hanımı bağışladı validem. Ben değilim [Müzik] kız. [Müzik]

🏆 Chunk 2 - Benzerlik: 0.120
----------------------------------------------------------------------
[Müzik] Evlat ile devlet [Müzik] terazinin iki kefesinde durur ise hangi yürek sarsılmaz? [Müzik] [Müzik] Bir yandan baba ocağının ateşı, öte yandan cihanın bekası. Hangisi yakar, hangisi yaşatır? Mevi gördüm ya Resulallah. [Müzik] Ölüm kimin alın yazısıdır? Öldürmek kimin kaderidir? Genç aslan her zaman mı yaşlı aslanın ensesinde olur? [Müzik] Kan ya evlat üçündür ya da devlet üçündür. Lakin her damlası, her damlası yüreğe ağır kalır

🏆 Chunk 3 - Benzerlik: 0.120
-----------------------------------

Hücre 8: RAG + Ollama Fonksiyonu

In [ ]:
def youtube_asistan(soru, collection, n_results=3):
    """
    YouTube video hakkında soru sor, Ollama ile cevap al
    """
    print("=" * 70)
    print(f"🔍 SORU: {soru}")
    print("=" * 70)
    
    # 1. RETRIEVAL
    print("\n📚 İlgili kısımlar aranıyor...")
    results = collection.query(
        query_texts=[soru],
        n_results=n_results
    )
    
    retrieved_docs = results['documents'][0]
    
    if not retrieved_docs:
        return "❌ İlgili bilgi bulunamadı."
    
    print(f"✅ {len(retrieved_docs)} ilgili kısım bulundu\n")
    
    # Bulunan chunk'ları göster
    for i, doc in enumerate(retrieved_docs):
        print(f"📄 Kaynak {i+1}: {doc[:80]}...")
    
    # 2. PROMPT OLUŞTUR
    context = "\n\n---\n\n".join(retrieved_docs)
    
    # Sorunun dilini tespit et
    soru_turkce = any(c in soru.lower() for c in ['ı', 'ğ', 'ü', 'ş', 'ö', 'ç']) or any(word in soru.lower() for word in ['ne', 'nedir', 'hangi', 'nasıl', 'kim', 'nerede', 'niye', 'niçin'])
    
    if soru_turkce:
        prompt = f"""Sen Türkçe konuşan bir video analiz asistanısın. Aşağıdaki video içeriğine dayanarak soruyu SADECE TÜRKÇE cevapla.

VİDEO İÇERİĞİ:
{context}

SORU: {soru}

TALİMATLAR:
- Cevabını SADECE TÜRKÇE yaz (başka dil kullanma!)
- Sadece video içeriğine dayanarak cevap ver
- Kısa ve öz ol
- Eğer bilgi yoksa "Bu bilgi videoda yok" de

TÜRKÇE CEVAP:"""
    else:
        prompt = f"""You are a video analysis assistant. Answer the question based ONLY on the video transcript below.

VIDEO TRANSCRIPT:
{context}

QUESTION: {soru}

INSTRUCTIONS:
- Answer in ENGLISH only
- Base your answer only on the transcript
- Be brief and concise
- If information is not in the transcript, say "This information is not in the video"

ANSWER:"""
    
    # 3. OLLAMA'YA GÖNDER
    print("\n🤖 Ollama cevap hazırlıyor...")
    
    try:
        response = ollama.generate(
            model='llama3.2',
            prompt=prompt
        )
        
        answer = response['response']
        
        print("\n" + "=" * 70)
        print("💬 CEVAP:")
        print("=" * 70)
        print(answer)
        
        return answer
        
    except Exception as e:
        error_msg = f"❌ Hata: {str(e)}\n\n💡 Ollama çalışıyor mu kontrol edin!"
        print(error_msg)
        return error_msg

print("✅ Fonksiyon hazır!")

✅ Fonksiyon hazır!


Hücre 9: Test Soruları

In [28]:
# TEST SORULARI

sorular = [
    "What is this video about?",
    "What topics are covered?",
    "Is Python explained in this video?",
]

for soru in sorular:
    youtube_asistan(soru, collection, n_results=3)
    print("\n" + "⏸️ " * 35 + "\n")

🔍 SORU: What is this video about?

📚 İlgili kısımlar aranıyor...
✅ 3 ilgili kısım bulundu

📄 Kaynak 1: . [Müzik] Sen şu kapıdan sağ salim girdin ya. Başkaca nimet istemem. Sultan baba...
📄 Kaynak 2: [Müzik] Evlat ile devlet [Müzik] terazinin iki kefesinde durur ise hangi yürek s...
📄 Kaynak 3: . >> Hey gafil bey kendini bilmez. Seni sancağa gönderdik ki düşmanın oyununu bo...

🤖 Ollama cevap hazırlıyor...

💬 CEVAP:
Bu video, Sultan II. Murad'ın kızının evliliği ile ilgili bir meseleye odaklanır. Sultan I. Murad'a ait bu video, 17. yüzyılda gerçekleşen olaylara dair bilgi vermektedir. Sultan I. Murad'ın kızının evlendirilmesiyle ilgili olarak video, sultanın kızının evli olmasındaki durumunu ve sultanın bu konudaki decisionine dair informações sunmaktadır.

⏸️ ⏸️ ⏸️ ⏸️ ⏸️ ⏸️ ⏸️ ⏸️ ⏸️ ⏸️ ⏸️ ⏸️ ⏸️ ⏸️ ⏸️ ⏸️ ⏸️ ⏸️ ⏸️ ⏸️ ⏸️ ⏸️ ⏸️ ⏸️ ⏸️ ⏸️ ⏸️ ⏸️ ⏸️ ⏸️ ⏸️ ⏸️ ⏸️ ⏸️ ⏸️ 

🔍 SORU: What topics are covered?

📚 İlgili kısımlar aranıyor...
✅ 3 ilgili kısım bulundu

📄 Kaynak 1: [Müzik] Evlat ile devlet

Hücre 10: Timestamp Bulma Özelliği (BONUS!)

In [ ]:
def find_timestamp(text_snippet, timestamps):
    """
    Verilen metin parçasının videoda hangi dakikada olduğunu bul
    """
    for ts in timestamps:
        if text_snippet[:50] in ts['text']:
            saniye = int(ts['start'])
            dakika = saniye // 60
            saniye = saniye % 60
            return f"{dakika}:{saniye:02d}"
    return "?"

def youtube_asistan_with_timestamp(soru, collection, timestamps, n_results=3):
    """
    YouTube video asistanı + timestamp gösterme
    """
    print("=" * 70)
    print(f"🔍 SORU: {soru}")
    print("=" * 70)
    
    # 1. RETRIEVAL
    print("\n📚 İlgili kısımlar aranıyor...")
    results = collection.query(
        query_texts=[soru],
        n_results=n_results
    )
    
    retrieved_docs = results['documents'][0]
    
    if not retrieved_docs:
        return "❌ İlgili bilgi bulunamadı."
    
    print(f"✅ {len(retrieved_docs)} ilgili kısım bulundu\n")
    
    # Bulunan chunk'ları timestamp ile göster
    print("📍 BULUNAN KISIMLAR:")
    print("-" * 70)
    for i, doc in enumerate(retrieved_docs):
        timestamp = find_timestamp(doc, timestamps)
        print(f"\n⏱️  [{timestamp}] - Kaynak {i+1}:")
        print(f"   {doc[:100]}...")
    
    # 2. PROMPT OLUŞTUR
    context = "\n\n---\n\n".join(retrieved_docs)
    
    # Sorunun dilini tespit et
    soru_turkce = any(c in soru.lower() for c in ['ı', 'ğ', 'ü', 'ş', 'ö', 'ç']) or any(word in soru.lower() for word in ['ne', 'nedir', 'hangi', 'nasıl', 'kim', 'nerede', 'niye', 'niçin'])
    
    if soru_turkce:
        prompt = f"""Sen Türkçe konuşan bir video analiz asistanısın. Aşağıdaki video içeriğine dayanarak soruyu SADECE TÜRKÇE cevapla.

VİDEO İÇERİĞİ:
{context}

SORU: {soru}

TALİMATLAR:
- Cevabını SADECE TÜRKÇE yaz (başka dil kullanma!)
- Sadece video içeriğine dayanarak cevap ver
- Kısa ve öz ol
- Eğer bilgi yoksa "Bu bilgi videoda yok" de

TÜRKÇE CEVAP:"""
    else:
        prompt = f"""You are a video analysis assistant. Answer the question based ONLY on the video transcript below.

VIDEO TRANSCRIPT:
{context}

QUESTION: {soru}

INSTRUCTIONS:
- Answer in ENGLISH only
- Base your answer only on the transcript
- Be brief and concise
- If information is not in the transcript, say "This information is not in the video"

ANSWER:"""
    
    # 3. OLLAMA'YA GÖNDER
    print("\n🤖 Ollama cevap hazırlıyor...")
    
    try:
        response = ollama.generate(
            model='llama3.2',
            prompt=prompt
        )
        
        answer = response['response']
        
        print("\n" + "=" * 70)
        print("💬 CEVAP:")
        print("=" * 70)
        print(answer)
        
        return answer
        
    except Exception as e:
        error_msg = f"❌ Hata: {str(e)}"
        print(error_msg)
        return error_msg

print("✅ Timestamp özellikli fonksiyon hazır!")

✅ Timestamp özellikli fonksiyon hazır!


Hücre 11: Timestamp'li Test

In [30]:
# TIMESTAMP İLE TEST

sorular = [
    "What is this video about?",
    "Are functions explained?",
    "What is covered in this tutorial?",
]

for soru in sorular:
    youtube_asistan_with_timestamp(soru, collection, timestamps, n_results=3)
    print("\n" + "⏸️ " * 35 + "\n")


🔍 SORU: What is this video about?

📚 İlgili kısımlar aranıyor...
✅ 3 ilgili kısım bulundu

📍 BULUNAN KISIMLAR:
----------------------------------------------------------------------

⏱️  [?] - Kaynak 1:
   . [Müzik] Sen şu kapıdan sağ salim girdin ya. Başkaca nimet istemem. Sultan baban seni bağışladı ya....

⏱️  [?] - Kaynak 2:
   [Müzik] Evlat ile devlet [Müzik] terazinin iki kefesinde durur ise hangi yürek sarsılmaz? [Müzik] [M...

⏱️  [?] - Kaynak 3:
   . >> Hey gafil bey kendini bilmez. Seni sancağa gönderdik ki düşmanın oyununu bozasın. Ama sen onlar...

🤖 Ollama cevap hazırlıyor...

💬 CEVAP:
Videodoran "Gafi" ismiyle tanıştırmış. Videodaki dil, style ve tone, 16. yüzyılda livedirlik çağındaki Türk halk kültürünü yansıtan bir dil gibi görünen eseri izliyor.

⏸️ ⏸️ ⏸️ ⏸️ ⏸️ ⏸️ ⏸️ ⏸️ ⏸️ ⏸️ ⏸️ ⏸️ ⏸️ ⏸️ ⏸️ ⏸️ ⏸️ ⏸️ ⏸️ ⏸️ ⏸️ ⏸️ ⏸️ ⏸️ ⏸️ ⏸️ ⏸️ ⏸️ ⏸️ ⏸️ ⏸️ ⏸️ ⏸️ ⏸️ ⏸️ 

🔍 SORU: Are functions explained?

📚 İlgili kısımlar aranıyor...
✅ 3 ilgili kısım bulundu

📍 BULUNAN KISIMLAR:
--------

Hücre 12: İnteraktif Sistem

In [31]:
def interaktif_youtube_asistan():
    """
    Kullanıcı ile interaktif konuşma
    """
    print("=" * 70)
    print("🎥 YOUTUBE VIDEO ASİSTANI")
    print("=" * 70)
    print("\nKomutlar:")
    print("  'yeni' - Yeni video analiz et")
    print("  'çıkış' - Programdan çık")
    print("=" * 70)
    
    while True:
        print("\n🎬 YouTube video linki girin (veya 'çıkış'):")
        video_input = input(">>> ").strip()
        
        if video_input.lower() in ['çıkış', 'cikis', 'exit', 'quit']:
            print("\n👋 Görüşmek üzere!")
            break
        
        # Video ID çıkar
        video_id = get_video_id(video_input)
        if not video_id:
            print("❌ Geçersiz YouTube linki!")
            continue
        
        # Transcript çek
        transcript_text, timestamps = get_transcript(video_id)
        if not transcript_text:
            print("❌ Transcript çekilemedi! Başka video deneyin.")
            continue
        
        # Chunk'la
        print("\n✂️  Chunking yapılıyor...")
        chunks = text_splitter.split_text(transcript_text)
        print(f"✅ {len(chunks)} chunk oluşturuldu")
        
        # Vector DB'ye ekle (yeni collection)
        print("\n💾 Vector DB'ye kaydediliyor...")
        try:
            client.delete_collection("youtube_db")
        except:
            pass
        
        collection = client.create_collection(
            name="youtube_db",
            embedding_function=sentence_transformer_ef
        )
        
        collection.add(
            documents=chunks,
            ids=[f"chunk_{i}" for i in range(len(chunks))],
            metadatas=[{"chunk_id": i, "video_id": video_id} for i in range(len(chunks))]
        )
        
        print("✅ Hazır! Sorularınızı sorabilirsiniz.\n")
        
        # Soru-cevap döngüsü
        while True:
            print("💬 Sorunuzu yazın ('yeni' ile başka video, 'çıkış' ile çık):")
            soru = input(">>> ").strip()
            
            if soru.lower() in ['çıkış', 'cikis', 'exit', 'quit']:
                print("\n👋 Görüşmek üzere!")
                return
            
            if soru.lower() in ['yeni', 'new']:
                print("\n🔄 Yeni video için hazır!")
                break
            
            if not soru:
                print("❌ Lütfen bir soru yazın!")
                continue
            
            # RAG ile cevap
            print("\n" + "⏳" * 35)
            youtube_asistan_with_timestamp(soru, collection, timestamps, n_results=3)

print("✅ İnteraktif sistem hazır!")
print("Başlatmak için sonraki hücreyi çalıştırın.")

✅ İnteraktif sistem hazır!
Başlatmak için sonraki hücreyi çalıştırın.


Hücre 13: Sistemi Başlat!

In [33]:
# İNTERAKTİF SİSTEMİ BAŞLAT
interaktif_youtube_asistan()


🎥 YOUTUBE VIDEO ASİSTANI

Komutlar:
  'yeni' - Yeni video analiz et
  'çıkış' - Programdan çık

🎬 YouTube video linki girin (veya 'çıkış'):
📥 Video ID: Gsg1msAj27U
⏳ Transcript çekiliyor...
✅ Transcript çekildi!
   📝 Dil: Türkçe
   ⏱️  Video süresi: 28 dakika 34 saniye
   📊 Toplam kelime: 4511
   📄 Karakter sayısı: 26566

✂️  Chunking yapılıyor...
✅ 66 chunk oluşturuldu

💾 Vector DB'ye kaydediliyor...
✅ Hazır! Sorularınızı sorabilirsiniz.

💬 Sorunuzu yazın ('yeni' ile başka video, 'çıkış' ile çık):

⏳⏳⏳⏳⏳⏳⏳⏳⏳⏳⏳⏳⏳⏳⏳⏳⏳⏳⏳⏳⏳⏳⏳⏳⏳⏳⏳⏳⏳⏳⏳⏳⏳⏳⏳
🔍 SORU: Bu videoda hangi yemekler yeniyor?

📚 İlgili kısımlar aranıyor...
✅ 3 ilgili kısım bulundu

📍 BULUNAN KISIMLAR:
----------------------------------------------------------------------

⏱️  [?] - Kaynak 1:
   . Bak bunu hiç fişek. Bir bu tas bu tas. Bir tas, bir örün zaten. >> H >> 8 >> 9. >> Özellikle yapın...

⏱️  [?] - Kaynak 2:
   . >> Bir yandan da lezzetli diyorsun. >> Dane. >> Dahe olsun. >> Allah'tan belanı mı istiyorsun? 10....

⏱️  [?] - K